In [1]:
import pandas as pd 
import numpy as np 
import seaborn as sns

In [ ]:
'''
    This Kernel is used to predict the missing values associated with Age from the test data. The problem with just 
    randomizing the data is that Age is shown to affect the survival outcome quite a bit as shown in the Data Analysis Kernel
    since Age is not going to be directly fed into the networks, rather the age group, a simple supervised neural network can 
    likely be used to some level of accuracy to predict the Age, which hopefully translates to the appropriate age group. 
    Note this method for sure has issues but should be a better approach than just simply guessing random values. 
    Please suggest better methods if possible 
    This next three jupyter notebook blocks are functions to feature some of the necessary values that I believe affect age
'''

In [2]:
def getTitle(value):
    if "Rev." in value:
        return 1
    elif "Col." in value:
        return 2
    elif "Mr." in value:
        return 3
    elif "Dr." in value:
        return 4
    elif "Master" in value:
        return 5
    elif "Miss" in value or "Mlle" in value:
        return 6
    elif "Mrs." in value or "Mme" in value:
        return 7
    else:
        return 0

In [3]:
def getAgeGroup(value):
    if value<=8:
        return 1
    if value>8 and value<=14:
        return 2
    if value >14 and value<=28:
        return 3
    if value >28 and value<=63:
        return 4
    if value >63:
        return 5

In [4]:
def getFareGroup(value):
    if value<=10:
        return 1
    if value>10 and value<=25:
        return 2
    if value >25 and value<=35:
        return 3
    if value >35 and value<=60:
        return 4
    if value >60 and value<=150:
        return 5
    if value > 150:
        return 6

In [ ]:
'''
    The data is read and cleaned according using the appropriate features for training the data on. 
'''

In [5]:
df=pd.read_csv(r'Raw_Data/train.csv')

In [6]:
df.index=df['PassengerId']
df.drop('PassengerId',axis=1,inplace=True)

In [7]:
df['male']=pd.get_dummies(df['Sex'])['male']

In [8]:
df['Q']=pd.get_dummies(df['Embarked'])['Q']
df['S']=pd.get_dummies(df['Embarked'])['S']

In [9]:
df['hasCabin']=df['Cabin'].apply(lambda x: 1 if type(x)==str else 0)

In [10]:
df=df[df['Age'].notnull()]

In [11]:
df['isChildRich']=0
df['isChildRich']=df['isChildRich'][df.Age<17][df.Pclass==1].replace(0,1)
df['isChildRich'].fillna(0,inplace=True)
df['isChildRich']=df['isChildRich'].apply(lambda x: int(x))

In [12]:
df['Age_Group']=df['Age'].apply(getAgeGroup)

In [13]:
df['Title']=df['Name'].apply(getTitle)

In [14]:
df['Fare_Group']=df['Fare'].apply(getFareGroup)

In [15]:
x=df[['Pclass','male','SibSp','Title','hasCabin','Fare_Group','Q','S']]
y=df['Age']

In [ ]:
'''
    A DNN is to be implemented, most of the hidden layers , batch size and epochs were chosen at random to minimize MSE, 
    didn't want to spend too much time on this doing a more indepth analysis. Seemed to not really give a better return. 
'''

In [16]:
from sklearn.model_selection import train_test_split

In [17]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.30,random_state=101)

In [18]:
import tensorflow as tf

C:\Users\Adam\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [19]:
feat_cols=[]

for col in x.columns:
    feat_cols.append(tf.feature_column.numeric_column(col))

In [20]:
input_func = tf.estimator.inputs.pandas_input_fn(x=x_train,y=y_train,batch_size=100,num_epochs=10000,shuffle=True)
Regression = tf.estimator.DNNRegressor(hidden_units=[400,200,100,50,25,12,6,3],feature_columns=feat_cols)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\Adam\\AppData\\Local\\Temp\\tmp_uz8vokb', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000000000E74ECC0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [21]:
Regression.train(input_fn=input_func,steps=10000)

Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Saving checkpoints for 0 into C:\Users\Adam\AppData\Local\Temp\tmp_uz8vokb\model.ckpt.
INFO:tensorflow:loss = 108500.12, step = 1
INFO:tensorflow:global_step/sec: 234.728
INFO:tensorflow:loss = 118041.92, step = 101 (0.428 sec)
INFO:tensorflow:global_step/sec: 280.096
INFO:tensorflow:loss = 106370.11, step = 201 (0.356 sec)
INFO:tensorflow:global_step/sec: 281.674
INFO:tensorflow:loss = 99841.15, step = 301 (0.354 sec)
INFO:tensorflow:global_step/sec: 281.674
INFO:tensorflow:loss = 92454.305, 

INFO:tensorflow:loss = 75764.96, step = 7001 (0.378 sec)
INFO:tensorflow:global_step/sec: 263.143
INFO:tensorflow:loss = 62974.625, step = 7101 (0.380 sec)
INFO:tensorflow:global_step/sec: 273.208
INFO:tensorflow:loss = 73503.984, step = 7201 (0.366 sec)
INFO:tensorflow:global_step/sec: 265.942
INFO:tensorflow:loss = 75440.52, step = 7301 (0.376 sec)
INFO:tensorflow:global_step/sec: 274.71
INFO:tensorflow:loss = 71449.664, step = 7401 (0.363 sec)
INFO:tensorflow:global_step/sec: 270.987
INFO:tensorflow:loss = 55901.22, step = 7501 (0.369 sec)
INFO:tensorflow:global_step/sec: 281.674
INFO:tensorflow:loss = 88560.5, step = 7601 (0.356 sec)
INFO:tensorflow:global_step/sec: 263.837
INFO:tensorflow:loss = 64299.88, step = 7701 (0.379 sec)
INFO:tensorflow:global_step/sec: 275.467
INFO:tensorflow:loss = 67640.55, step = 7801 (0.363 sec)
INFO:tensorflow:global_step/sec: 278.535
INFO:tensorflow:loss = 65459.16, step = 7901 (0.359 sec)
INFO:tensorflow:global_step/sec: 264.535
INFO:tensorflow:los

In [22]:
pred_fn = tf.estimator.inputs.pandas_input_fn(x=x_test,batch_size=len(x_test),shuffle=False)

In [23]:
note_predictions = list(Regression.predict(input_fn=pred_fn))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\Adam\AppData\Local\Temp\tmp_uz8vokb\model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [24]:
final_preds  = []
for pred in note_predictions:
    final_preds.append(pred['predictions'][0])

In [25]:
from sklearn.metrics import mean_squared_error

In [26]:
mse = mean_squared_error(final_preds, y_test)

In [27]:
mse

612.3424942955535

In [28]:
compare=pd.DataFrame(index=y_test.index)
compare['Age']=y_test
compare['Predict']=final_preds

In [291]:
from sklearn.linear_model import LinearRegression

In [292]:
linear_model=LinearRegression()

In [293]:
linear_model.fit(x_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [294]:
linear_predict=linear_model.predict(x_test)

In [295]:
compare['linear_predict']=list(linear_predict)

In [296]:
len(x_train.columns)

8

In [297]:
print(compare.to_string())

               Age    Predict  linear_predict
PassengerId                                  
854          16.00  31.066093       38.235035
541          36.00  30.649996       37.207756
260          50.00  34.626423       31.282573
691          31.00  42.840084       37.087096
470           0.75   5.215958       17.549400
871          26.00  28.672651       28.316448
58           28.50  27.500542       26.293057
708          42.00  49.119324       41.069147
449           5.00   5.215958       17.549400
294          24.00  23.947290       26.509615
418          18.00  29.333609       34.183749
176          18.00  25.737272       25.361675
284          19.00  28.672651       28.316448
766          51.00  35.622574       32.379087
427          28.00  32.771599       28.327801
41           40.00  32.298187       21.680945
105          37.00  24.855938       22.406903
312          18.00  21.998629       28.247542
393          28.00  24.855938       22.406903
545          50.00  33.658630     

In [29]:
compare['Predict_Age_Group']=compare['Predict'].apply(getAgeGroup)

In [30]:
compare['Truee_Age_Group']=compare['Age'].apply(getAgeGroup)

In [31]:
#Quite a low MSE, although an age group miss of 1 can result in a large age difference, the network seemed to guess 
#the appropriate age groups for low and high ages. The middle values hopefully won't make a difference
len(compare[compare['Predict_Age_Group']==compare['Truee_Age_Group']])/len(compare)

0.027906976744186046

In [ ]:
'''
    Simply applying the trained network 
'''

In [33]:
predict_age=pd.read_csv(r'Raw_Data/test.csv')

In [34]:
predict_age.index=predict_age['PassengerId']
predict_age.drop('PassengerId',axis=1,inplace=True)
predict_age['male']=pd.get_dummies(predict_age['Sex'])['male']
predict_age['Q']=pd.get_dummies(predict_age['Embarked'])['Q']
predict_age['S']=pd.get_dummies(predict_age['Embarked'])['S']
predict_age['hasCabin']=predict_age['Cabin'].apply(lambda x: 1 if type(x)==str else 0)

In [35]:
predict_age['isChildRich']=0
predict_age['isChildRich']=predict_age['isChildRich'][predict_age.Age<17][predict_age.Pclass==1].replace(0,1)
predict_age['isChildRich'].fillna(0,inplace=True)
predict_age['isChildRich']=predict_age['isChildRich'].apply(lambda x: int(x))
predict_age['Title']=predict_age['Name'].apply(getTitle)

In [36]:
predict_age['Fare_Group']=predict_age['Fare'].apply(getFareGroup)
x_age_predict=predict_age[['Pclass','male','SibSp','Title','hasCabin','Fare_Group','Q','S']]

In [37]:
age_pred_fn = tf.estimator.inputs.pandas_input_fn(x=x_age_predict,batch_size=len(x_age_predict),shuffle=False)
age_note_predictions = list(Regression.predict(input_fn=age_pred_fn))
age_final_preds  = []
for pred in age_note_predictions:
    age_final_preds.append(pred['predictions'][0])

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\Adam\AppData\Local\Temp\tmp_uz8vokb\model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [38]:
comparison_func=pd.read_csv(r'Raw_Data/test.csv')
comparison_func['Age_Predict']=age_final_preds
comparison_func=comparison_func[['Age','Age_Predict']]

In [39]:
comparison_func['Age_Final']=comparison_func['Age']
comparison_func['Age_Final'][comparison_func['Age'].isnull()]=comparison_func['Age_Predict'][comparison_func['Age'].isnull()]
comparison_func['Age_Final']=round(comparison_func['Age_Final'],0)

In [40]:
test_age_updated=pd.read_csv(r'Raw_Data/test.csv')
test_age_updated['Age']=comparison_func['Age_Final']
test_age_updated.index=test_age_updated['PassengerId']
test_age_updated.drop('PassengerId',axis=1,inplace=True)
test_age_updated.to_csv('Clean_Data/test.csv', encoding='utf-8', index=True)